# Point Compositions

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import scienceplots
import pandas as pd
from itertools import combinations
import pprint
from shapely.geometry import Polygon
plt.style.use(['science', 'grid', 'ieee'])

from utils import (
    load_dataset,
    get_images,
    transform_points,
    get_densities,
    predict,
    calculate_errors,
    get_result_dict,
    SCALING_FACTOR,
    plot_setup, 
    plot_setup_noised,
    plot_predictions,
    add_outlier
)

def filter_points(points, keys):
    filtered = {key: points[key] for key in keys}
    return filtered

def conduct_experiment(pv_img, tv_img, plot=True, print_errors=True, used_ref_points=None, return_homography=False):
    # Load Data
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
    img_pv, img_tv = get_images(pv_img, tv_img)
    
    if used_ref_points:
        reference_pts_pv = filter_points(reference_pts_pv, used_ref_points)
        reference_pts_tv = filter_points(reference_pts_tv, used_ref_points)
        
    # Transform Points to Homogeneous Numpy arrays
    reference_pts_pv_arr, reference_pts_tv_arr, validation_pts_pv_arr, validation_pts_tv_arr = transform_points(
        reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv)
    # Calculate Homography
    h, _ = cv2.findHomography(
        reference_pts_pv_arr,
        reference_pts_tv_arr,
        # method = cv2.RANSAC,
        method = 0,
    )
    h_inv = np.linalg.inv(h)

    # Get Pixel Densities
    reference_densities = get_densities(reference_pts_tv_arr, h_inv)
    validation_densities = get_densities(validation_pts_tv_arr, h_inv)
    
    # Predict Points
    predicted_reference_pts_tv = predict(reference_pts_pv_arr, h)
    predicted_validation_pts_tv = predict(validation_pts_pv_arr, h)
    
    # Errors
    reference_errors = calculate_errors(predicted_reference_pts_tv, reference_pts_tv_arr)
    validation_errors = calculate_errors(predicted_validation_pts_tv, validation_pts_tv_arr)

    # Combine Results in Dictionary
    reference_result_dict = get_result_dict(reference_pts_pv, reference_pts_tv_arr, predicted_reference_pts_tv, reference_errors, reference_densities, reference_pts_pv_arr)
    validation_result_dict = get_result_dict(validation_pts_pv, validation_pts_tv_arr, predicted_validation_pts_tv, validation_errors, validation_densities, reference_pts_pv_arr)
    
    # Print Errors
    if print_errors:
        print(f'Reference  Error: {np.mean(reference_errors) :.2f} PX!')
        print(f'Reference  Error: {np.mean(reference_errors) / SCALING_FACTOR * 100 :.2f} CM!')
        print('---')
        print(f'Validation Error: {np.mean(validation_errors):.2f} PX!')
        print(f'Validation Error: {np.mean(validation_errors) / SCALING_FACTOR * 100 :.2f} CM!')
    
    # Plots
    if plot:
        plot_setup(img_tv, img_pv, reference_result_dict, validation_result_dict)
        plot_predictions(img_tv, img_pv, reference_result_dict, validation_result_dict)
    if return_homography:
        return reference_result_dict, validation_result_dict, h
    return reference_result_dict, validation_result_dict

def get_collinearity_score(subset, reference_pts_pv):
    A = np.array(reference_pts_pv[subset[0]])
    B = np.array(reference_pts_pv[subset[1]])
    C = np.array(reference_pts_pv[subset[2]])
    AB = np.linalg.norm(A-B) 
    AC = np.linalg.norm(A-C) 
    BC = np.linalg.norm(B-C)
    distances = sorted([AB, AC, BC])
    return (distances[0] + distances[1] - distances[2]) / distances[2] * 100
    
def get_overall_collinearity_score(used_ref_pts, reference_pts_pv):
    scores = []
    for subset in combinations(used_ref_pts, 3):
        score = get_collinearity_score(subset, reference_pts_pv)
        scores.append(score)
    return min(scores)

def get_collinearity_scores(used_ref_pts, reference_pts_pv):
    scores = []
    for subset in combinations(used_ref_pts, 3):
        score = get_collinearity_score(subset, reference_pts_pv)
        scores.append(score)
    return scores

## How Many Points?

Using four points is enough to determine a homography matrix $\mathrm{H}$. However, as we have noise, we can use more points and create an overdetermined system of linear equations. Then, we can use an optimization technique to calculate $\mathrm{H}$.

How many points should we use? Let's see.

Let's consider image DJI_0026 and test various point compositions of **four**, **five**, **six**, **seven**, **eight**, **nine** reference points. Let's start by doing the experiments completely random, and then filter degenerate solutions; see **2.2.1 Collinearity**. To confirm our conclusion, do the same experiment for DJI_0029.

In [ ]:
def n_point_experiment(pv_img, tv_img):
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
    all_reference_points = list(reference_pts_pv.keys())

    rows = []
    # For all reference point combinations with k=4
    for k in range(4, len(all_reference_points)+1):
        print('---'*10, k, '--'*10)
        for i, subset in enumerate(combinations(all_reference_points, k)):
            print('#'*10)
            print(f'Experiment {i}')
            # Conduct Experiment
            reference_result_dict, validation_result_dict = conduct_experiment(pv_img, tv_img, used_ref_points=subset, plot=False)

            spanning_points = []
            for ref_pt_name in subset:
                spanning_points.append(reference_pts_pv[ref_pt_name])
            polygon = Polygon(spanning_points)
            spanned_area = polygon.area / (4000*3000) * 100
    
            # Make resulting Dict 
            collinearity_score = get_overall_collinearity_score(subset, reference_pts_pv)
            collinearity_scores = get_collinearity_scores(subset, reference_pts_pv)
            for pt, pt_dict in validation_result_dict.items():
                pt_dict['k'] = k
                pt_dict['experiment'] = i
                pt_dict['name'] = pt
                pt_dict['collinearity'] = collinearity_score
                pt_dict['collinearity_scores'] = collinearity_scores
                pt_dict['used_ref_pts'] = subset
                pt_dict['spanned_area'] = spanned_area
                rows.append(pt_dict)
    df = pd.DataFrame(rows)
    df['img'] = pv_img
    return df
n_pt_results = n_point_experiment('IMG_01', 'IMG_00')

In [ ]:
fig, ax = plt.subplots(1, 1)#, figsize=(12, 10))
n_pt_results.boxplot(
    column='error',
    by='k',
    showfliers=True,
    vert=False,
    patch_artist=True,
    boxprops=dict(facecolor='lightgray', lw=1),
    medianprops=dict(color='black', lw=2),
    whiskerprops = dict(color = "black", lw=1),
    widths=0.5,
    ax=ax
)
plt.show()

In [ ]:
inliers = n_pt_results.loc[((n_pt_results['collinearity'] >= 1) & (n_pt_results['k'] == 4)) | (n_pt_results['k'] != 4)] # TODO!
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
inliers.boxplot(
    column='error',
    by='k',
    showfliers=True,
    vert=False,
    patch_artist=True,
    boxprops=dict(facecolor='lightgray', lw=1),
    medianprops=dict(color='black', lw=2),
    whiskerprops = dict(color = "black", lw=1),
    widths=0.5,
    ax=ax1
)
inliers.boxplot(
    column='error',
    by='k',
    showfliers=False,
    vert=False,
    patch_artist=True,
    boxprops=dict(facecolor='lightgray', lw=1),
    medianprops=dict(color='black', lw=2),
    whiskerprops = dict(color = "black", lw=1),
    widths=0.5,
    ax=ax2
)
fig.suptitle('')
ax1.set_title('')
plt.show()

In [ ]:
inliers = n_pt_results.loc[((n_pt_results['collinearity'] >= 1) & (n_pt_results['k'] == 4)) | (n_pt_results['k'] != 4)] # TODO!
fig, ax1 = plt.subplots(1, 1)#, figsize=(10, 10))
inliers.boxplot(
    column='error',
    by='k',
    showfliers=False,
    vert=False,
    patch_artist=True,
    boxprops=dict(facecolor='lightgray', lw=.8),
    medianprops=dict(color='black', lw=1),
    whiskerprops = dict(color = "black", lw=.8),
    widths=0.5,
    ax=ax1
)
fig.suptitle('')
ax1.set_title('error [px]')
# ax1.set_xlabel('error [px]')
ax1.set_ylabel('number of reference points $k$')

t1 = ax1.text(13.5, 1.3, 'Outlier until $122.6$', fontsize=6)
t1.set_bbox(dict(facecolor='white', alpha=1, edgecolor='black', lw=.5))
t2 = ax1.text(13.5, 1.97, 'Outlier until $75.8$', fontsize=6)
t2.set_bbox(dict(facecolor='white', alpha=1, edgecolor='black', lw=.5))
t3 = ax1.text(13.5, 2.97, 'Outlier until $35.7$', fontsize=6)
t3.set_bbox(dict(facecolor='white', alpha=1, edgecolor='black', lw=.5))
plt.grid(False)
plt.savefig('output/boxplot_errors_by_k_filtered.png', dpi=600)
plt.show()

In [ ]:
# collinearity scores with k=5??
fives = n_pt_results.loc[n_pt_results['k'] == 5]
fives

In [ ]:
fives.plot.scatter(
    x='collinearity', 
    y='error',
    s=100,
    c='distance_to_next_ref_pt',
    colormap='viridis',
    alpha=0.8,
    figsize=(12, 10)
)
plt.show()

In [ ]:
fives.plot.scatter(
    x='spanned_area', 
    y='error',
    s=100,
    c='distance_to_next_ref_pt',
    colormap='viridis',
    alpha=0.8,
    figsize=(12, 10)
)
plt.show()

In [ ]:
inliers.plot.scatter(
    x='spanned_area', 
    y='error',
    s=100,
    c='distance_to_next_ref_pt',
    colormap='viridis',
    alpha=0.8,
    figsize=(12, 10)
)
plt.savefig('scatter_error_by_area_distance.png', dpi=300)
plt.show()

In [ ]:
inliers_spanned_area = inliers.loc[inliers['spanned_area'] >= 10] 
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
inliers_spanned_area.boxplot(
    column='error',
    by='k',
    showfliers=True,
    vert=False,
    patch_artist=True,
    boxprops=dict(facecolor='lightgray', lw=1),
    medianprops=dict(color='black', lw=2),
    whiskerprops = dict(color = "black", lw=1),
    widths=0.5,
    ax=ax
)
plt.show()